In [1]:
# imports

# !pip install tf-models-official
import tqdm
import tensorflow as tf
# import tensorflow_models as tfm
import numpy as np
import json
import os


2024-09-12 19:57:33.882835: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 19:57:33.983444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1726163854.024555    6960 cuda_dnn.cc:8322] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1726163854.036379    6960 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 19:57:34.130326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# download the dataset

# !wget http://images.cocodataset.org/zips/train2014.zip
# !wget http://images.cocodataset.org/zips/val2014.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
# !wget http://images.cocodataset.org/zips/test2014.zip
# !wget http://images.cocodataset.org/annotations/image_info_test2014.zip


In [ ]:
# unzip the dataset files

# !pip install unzip
# !unzip /home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014.zip


In [2]:
# dataset paths

train_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/train2014/"
val_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014/"
test_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/test2014/"
annotations = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/annotations/"

In [3]:
# loading and procseing the dataset

annotation_path = os.path.join(annotations, "instances_val2014.json")
with open(annotation_path, "r") as f:
    json_obj = json.load(f)
    print(json_obj.keys())

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])


In [2]:
tfm.vision.decoders.FPN(
    input_specs: Mapping[str, tf.TensorShape],
    min_level: int = 3,
    max_level: int = 7,
    num_filters: int = 256,
    fusion_type: str = 'sum',
    use_separable_conv: bool = False,
    use_keras_layer: bool = False,
    activation: str = 'relu',
    use_sync_bn: bool = False,
    norm_momentum: float = 0.99,
    norm_epsilon: float = 0.001,
    kernel_initializer: str = 'VarianceScaling',
    kernel_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    bias_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    **kwargs
)

SyntaxError: invalid syntax (2388210152.py, line 2)